In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Feature Extraction (Handcrafted and Deep Features)

import cv2
import numpy as np
import pandas as pd
import re
import math
from skimage import feature
from skimage.feature import hog
from imutils import paths
from google.colab.patches import cv2_imshow
import os

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input

# Load Inception_v3 pretrained on ImageNet dataset
model = InceptionV3(include_top=False, weights='imagenet', pooling='avg', input_tensor=Input(shape=(299,299,3)))
#model = Xception(include_top=False, weights='imagenet', pooling='avg')

# List of paths 
import os
file_list=[]
file_list.append(os.listdir(r"/content/drive/My Drive/Base1/humanos"))
file_list.append(os.listdir(r"/content/drive/My Drive/Base1/praia"))
file_list.append(os.listdir(r"/content/drive/My Drive/Base1/obras"))
file_list.append(os.listdir(r"/content/drive/My Drive/Base1/onibus"))
file_list.append(os.listdir(r"/content/drive/My Drive/Base1/dino"))
file_list.append(os.listdir(r"/content/drive/My Drive/Base1/elefante"))
file_list.append(os.listdir(r"/content/drive/My Drive/Base1/flores"))
file_list.append(os.listdir(r"/content/drive/My Drive/Base1/cavalos"))
file_list.append(os.listdir(r"/content/drive/My Drive/Base1/montanhas"))
file_list.append(os.listdir(r"/content/drive/My Drive/Base1/comida"))

# general path
path='/content/drive/My Drive/Base1/'

# list of classes
class_names=['humanos', 'praia', 'obras', 'onibus', 'dino', 'elefante', 'flores', 'cavalos', 'montanhas', 'comida'] 
#class_names=['dogs', 'cats']

X_deep = []
y = []

# Feature extraction
for classes_files, classe in zip (file_list, range(10)):
    for i in range(100):
      name= str(path) + str(class_names[classe]) + str('/') + str(classes_files[i]) 
      print(name)
      y.append(classe)
      
# Extract deep features using InceptionV3 pretrained model 
      imagem = cv2.imread(name)
      img = cv2.resize(imagem,(299,299))
      xd = image.img_to_array(img)
      xd = np.expand_dims(xd, axis=0)
      xd = preprocess_input(xd)
      deep_features = model.predict(xd)
      print(deep_features.shape)
      
      X_image_aux = []
      for aux in deep_features:
          X_image_aux = np.append(X_image_aux, np.ravel(aux))
    
      deep_features = [i for i in X_image_aux]
      
      X_deep.append(deep_features)

# Saving the extracted features (deep) in a csv file
df = pd.DataFrame(X_deep)
df.to_csv('X_deep.csv', header=False, index=False)

# Saving the classes in a csv file
df_class = pd.DataFrame(y)
df_class.to_csv('y.csv', header=False, index=False)

In [ ]:
# Example: how to load the csv files (features and labels)

import pandas as pd
import numpy as np

# Labels
y = pd.read_csv('y.csv', header=None)
y=y.to_numpy()
y=np.ravel(y)
print(y.shape)

# deep features
X = pd.read_csv('X_deep.csv', header=None)
X=X.to_numpy()
print(X.shape)

In [ ]:
# Treinamento de classificadores

import numpy as np
import urllib
from sklearn.naive_bayes import GaussianNB
from sklearn import  model_selection
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

# EXEMPLO USANDO HOLDOUT
# Holdout -> dividindo a base em treinamento (60%) e teste (40%), estratificada
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.4, random_state=42, stratify=y)

# Treina o classificador

clfa = GaussianNB()
clfa = clfa.fit(X_train, y_train)

# testa usando a base de testes
predicted=clfa.predict(X_test)
predp=clfa.predict_proba(X_test)

# calcula a acurÃ¡cia na base de teste
score=clfa.score(X_test, y_test)

# calcula a matriz de confusÃ£o
matrix = confusion_matrix(y_test, predicted)

# apresenta os resultados
print("Accuracy = %.2f " % score)
print("Confusion Matrix:")
print(matrix)

# EXEMPLO USANDO VALIDAÃ‡ÃƒO CRUZADA

clfb = GaussianNB()

folds=10
result = model_selection.cross_val_score(clfb, X, y, cv=folds)
print("\nCross Validation Results %d folds:" % folds)
print("Mean Accuracy: %.2f" % result.mean())
print("Mean Std: %.2f" % result.std())

# matriz de confusÃ£o da validaÃ§Ã£o cruzada
Z = model_selection.cross_val_predict(clfb, X, y, cv=folds)
cm=confusion_matrix(y, Z)
print("Confusion Matrix:")
print(cm)

In [ ]:
# Plot mistakes (images)
print(predicted.shape)
for i in range(len(predicted)):
    if (predicted[i] != y_test[i]):
        dist=1
        j=0
        while (j<len(X) and dist !=0): 
            dist = np.linalg.norm(X[j]-X_test[i])
            j+=1
        print("Label:", y[j-1], class_names[y[j-1]], "  /  Prediction: ", predicted[i], class_names[predicted[i]], predp[i][predicted[i]])
        name= "/content/drive/My Drive/Base1/"+ str(class_names[y[j-1]]) + "/" + str(j)+ ".jpg" 
        print(name)
        im=cv2.imread(name)
        cv2_imshow(im)
        print("=============================================================================")
  
